In [1]:
import multiprocessing
import pycurl
import pymysql
import urllib.parse
import io
import re
import json
import requests
from lxml.html import fromstring, tostring

In [2]:
timeout=60
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36'}

In [3]:
def try_catch(func):
    def wrapper(*args, **kw):
        try:
            func(*args, **kw)
        except Exception as e:
            pmid = args[0]
            with open('./errors/'+ str(pmid) + '.html', 'w') as f:
                f.write(text)
            cu.execute('insert into journal_error values(%s, %s)', (pmid, str(e)))
            con.commit()
#         return func(*args, **kw)
    return wrapper

In [4]:
con = pymysql.connect(host='localhost', user='root', passwd='', db='xiaobaifinder', charset='utf8')
cu_r = con.cursor()
cu = con.cursor()
cu_r.execute('select link from doi_link where link like "%oxfordjournals%" limit 20')
link_list = cu_r.fetchall()
for link in link_list:
    print(link[0])
cu_r.close()
cu.close()
con.close()

http://bioinformatics.oxfordjournals.org/cgi/doi/10.1093/bioinformatics/bth468
http://bioinformatics.oxfordjournals.org/cgi/doi/10.1093/bioinformatics/bth486
http://bioinformatics.oxfordjournals.org/cgi/doi/10.1093/bioinformatics/bti090
http://jxb.oxfordjournals.org/lookup/doi/10.1093/jxb/eri013
http://ndt.oxfordjournals.org/cgi/doi/10.1093/ndt/gfh574
http://humrep.oxfordjournals.org/lookup/doi/10.1093/humrep/deh583
http://humrep.oxfordjournals.org/lookup/doi/10.1093/humrep/deh594
http://bioinformatics.oxfordjournals.org/cgi/doi/10.1093/bioinformatics/bti197
http://bioinformatics.oxfordjournals.org/cgi/doi/10.1093/bioinformatics/bti211
http://humrep.oxfordjournals.org/lookup/doi/10.1093/humrep/deh676
http://humrep.oxfordjournals.org/lookup/doi/10.1093/humrep/deh668
http://cardiovascres.oxfordjournals.org/cgi/doi/10.1016/j.cardiores.2004.09.027
http://cardiovascres.oxfordjournals.org/cgi/doi/10.1016/j.cardiores.2004.09.020
http://femsle.oxfordjournals.org/cgi/doi/10.1016/j.femsle.2004.1

In [ ]:
url = link_list[0]
print(url)
text = requests.get(url, headers = headers,timeout=timeout).content.decode('utf-8')


In [4]:
def tandfonline(pmid,url):
    text = requests.get(url, headers = headers,timeout=timeout).content.decode('utf-8')
    try:
        root = fromstring(text)
        author_index = 0            
        for author_li in root.cssselect('span.hlFld-ContribAuthor'):
    #         print(tostring(author_li))
            if 'NLM_degrees' in author_li.get('class'):
                continue;
            corresp=0
            author=None
            author_index +=1
            author_aff_list=[]
            author_aff_name=None
            email=None
            author = author_li.find('a').text_content().strip()
    #             print(author)
    #         author = re.compile('[0-9]').sub('', author)
            sup = author_li
            for ii in range(5):
                if sup  and '*' in sup.text_content():
                    corresp = 1
                elif sup:
                    sup = sup.getnext()
                else:
                    break
                if sup and sup.tag and sup.tag == 'span':
                    break        
    #         print(pmid, author_index, author, author_aff_name, email, corresp)
    #         cu.execute('insert into journal_author values(%s, %s, %s, %s, %s, %s)',
    #            (pmid, author_index, author, author_aff_name, email, corresp,))
    #         con.commit() 
        # keywords
        keywords = [kw.find('a').text for kw in  root.cssselect('ul.keywords li')]
        for kw in keywords:
            if kw:
                cu.execute('insert into journal_keyword values(%s, %s)', (pmid, kw.replace(';', '').strip(),))
                con.commit()
    except Exception as e:
        with open('./errors/'+ str(pmid) + '.html', 'w') as f:
            f.write(text)
        cu.execute('insert into journal_error values(%s, %s)', (pmid, str(e)))
        con.commit()    
        

In [5]:
def rsc(pmid, url):
    text = requests.get(url, headers = headers,timeout=timeout).content.decode('utf-8')
    try:
        root = fromstring(text)
        # aff
        aff_dict={}
        aff_index = 0
        aff_list = root.cssselect('div.show_aff_content')
        corresp_email=[]
        for aff in aff_list:
            aff_index +=1
            if aff.find('div').text.strip() == '*':
                aff_index -=1
                continue
            email = ''
            email_div = aff.findall('div')[1].find('a')
            if email_div is not None and email_div.get('title'):
                email =  email_div.get('title')
                corresp_email.append(email)

            sup = aff.cssselect('sup.affiliation_sup')[0].text_content().strip()
    #             print(sup)
            aff_name = aff.findall('div')[1].text.replace('E-mail:', '').replace(email, '').strip()

            # 查找email
            aff_dict[sup] = aff_name        
            cu.execute('insert into journal_aff (pmid, aff_index, aff_name) values(%s, %s, %s)', (pmid, aff_index, aff_name,))
            con.commit()
    ## author
        author_index = 0
        for author_li in root.cssselect('span.author_link'):
    #         print(tostring(author_li))
            corresp=0
            author=None
            email = None
            author_aff_name = None
            author_index +=1
            author_aff_list=[]
            author = author_li.find('a').text_content().strip()
    #         author = re.compile('[0-9]').sub('', author)

            sups = author_li.findall('sup')
            for sup in sups:
                aff_name = aff_dict[sup.text]
                author_aff_list.append(aff_name)

            authro_corresp = author_li.find('span')
    #         print(tostring(authro_corresp[0]).text_content)
            if authro_corresp is not None and '*' in authro_corresp.text_content():

                corresp=1
                email = corresp_email.pop(0)
            for author_aff_name in author_aff_list:
    #             print(pmid, author_index, author, author_aff_name, email, corresp)
                cu.execute('insert into journal_author values(%s, %s, %s, %s, %s, %s)',
                   (pmid, author_index, author, author_aff_name, email, corresp,))
                con.commit() 
    # keywords
    except Exception as e:
        with open('./errors/'+ str(pmid) + '.html', 'w') as f:
            f.write(text)
        cu.execute('insert into journal_error values(%s, %s)', (pmid, str(e)))
        con.commit()


In [6]:
#coding=utf-8
def acs(pmid, url):
    text = requests.get(url, headers = headers,timeout=timeout).content.decode('utf-8')

    try:
        root = fromstring(text)
        # aff
        aff_dict={}
        aff_index = 0
        aff_list = root.cssselect('div.affiliations div')
        first = None
        if len(aff_list)==1:
            first=1
        for aff in aff_list:
            aff_index +=1

            if not first and aff.find('sup'):
                sup = aff.find('sup').text_content()
#                 print(sup)
                aff_name = aff.text_content().replace(sup, '').replace('\r\n', '').strip()
                aff_dict[sup] = aff_name
#                 print(aff_name)
            else:
                aff_name = aff.text_content().replace('\r\n', '').strip()
                aff_dict['first']=aff_name
                cu.execute('insert into journal_aff (pmid, aff_index, aff_name) values(%s, %s, %s)', (pmid, aff_index, aff_name,))
                con.commit()
        corresp_email = [email.text for email in root.cssselect('div#correspondence a')]
        # author
        author_index=0

        equal = re.compile('Authors with equal contribution').findall(text)

        for author_li in root.cssselect('div#authors>span.hlFld-ContribAuthor'):
    #         print(tostring(author_li))
            corresp=0
            author=None
            email = None
            author_aff_name = None
            author_index +=1
            author_aff_list=[]
            author = author_li.text_content().replace(',', '').replace('and', '').replace('*', '').strip()
            author = re.compile('[0-9]').sub('', author)
            if not first:
                sups = author_li.findall('a.NLM_xref-aff')
                for sup in sups:
                    aff_name = aff_dict[sup.text]
                author_aff_list.append(aff_name)
            else:
                author_aff_list.append(aff_dict['first'])
            authro_corresp = author_li.cssselect('a.ref')
    #         print(tostring(authro_corresp[0]).text_content)
            if authro_corresp and authro_corresp[0].text_content() == '*':

                corresp=1
                if corresp_email:
                    email = corresp_email.pop(0)
            for author_aff_name in author_aff_list:
    #             pass
#                     print(pmid, author_index, author, author_aff_name, email, corresp)
                    cu.execute('insert into journal_author values(%s, %s, %s, %s, %s, %s)',
                       (pmid, author_index, author, author_aff_name, email, corresp,))
                    con.commit() 
        # keywords
        keywords = [kw.text for kw in  root.cssselect('div.keywords a')]
#         print(keywords)
        for kw in keywords: 
            if kw:
#                 print(kw.strip())
                cu.execute('insert into journal_keyword values(%s, %s)', (pmid, kw.strip(),))
                con.commit()
    except Exception as e:
        with open('./errors/'+ str(pmid) + '.html', 'w') as f:
            f.write(text)
        cu.execute('insert into journal_error values(%s, %s)', (pmid, str(e)))
        con.commit()

In [7]:
def wiley(pmid, url):
    # wiley 存在问题， 个别时候 * 即表示共同第一又表示通讯, 已解决
    text = requests.get(url, headers = headers,timeout=timeout).content.decode('utf-8')
    try:
        root = fromstring(text)
        new = root.cssselect('a#wol1backlink')
        if new:
#             print(new)
            url= new[0].get('href')
            text = requests.get(url, headers = headers,timeout=timeout).content.decode('utf-8')
            root = fromstring(text)
        # aff
#authorsAffiliations---------
#         if not isinstance(text, str):
#             text = text.decode('utf-8')
        aff_dict={}
        aff_index = 0
        aff_list = root.cssselect('ol#authorsAffiliations li')
        first = None # 单位的数目
        if len(aff_list)==1:
            first=1
        for aff in aff_list:
            aff_index +=1
            aff_name = aff.find('p').text_content()
            if not first:
                sup = aff.find('span').text_content()
                aff_dict[sup] = aff_name
            else:
                aff_dict['first']=aff_name
            cu.execute('insert into journal_aff (pmid, aff_index, aff_name) values(%s, %s, %s)', (pmid, aff_index, aff_name,))
            con.commit()
        corresp_email = [email.text for email in root.cssselect('p#correspondence a')]
        # author
        author_index=0

        equal = re.compile('Authors with equal contribution').findall(text)

        for author_li in root.cssselect('ol#authors li'):
#             print(tostring(author_li))
            corresp=0
            author=None
            email = None
            author_aff_name = None
            author_index +=1
            author = author_li.text_content().replace(',', '').replace('and', '').replace('*', '').strip()
            author = re.compile('[0-9]').sub('', author)
            author_aff_list = []
            author_sup = author_li.find('sup')
            if author_sup is not None: 
                sups = author_sup.text_content().split(',')
#                 print(sups)
                for sup in sups:
                    if sup in  aff_dict:# 标号是不是在sups里面
                        author_aff_list.append(aff_dict[sup])
    #                 elif first:
    #                     author_aff_list.append(aff_dict['first'])
                    if (sup=='*' and equal and (author_index>2)) or (not equal and sup=='*') : # 解决共同第一的问题
                        corresp=1
                        if corresp_email:
                            email = corresp_email.pop(0)
#                         print(email,equal)
            if first:#没有 sup也就是只有一个单位
                author_aff_list.append(aff_name)

            for author_aff_name in author_aff_list:
                cu.execute('insert into journal_author values(%s, %s, %s, %s, %s, %s)',
                   (pmid, author_index, author, author_aff_name, email, corresp,))
#                 print(pmid, author_index, author, author_aff_name, email, corresp,)
                con.commit()

        #keyword
        for kw in root.cssselect('ul.keywordList li'):
    #         print(kw.text_content().replace(';', '').strip())
            cu.execute('insert into journal_keyword values(%s, %s)', (pmid, kw.text_content().replace(';', '').strip(),))
            con.commit()
    except Exception as e:
        with open('./errors/'+ str(pmid) + '.html', 'w') as f:
            f.write(text)
        cu.execute('insert into journal_error values(%s, %s)', (pmid, str(e)))
        con.commit()

    

In [8]:
# url = 'http://link.springer.com/article/10.1007%2Fs12149-015-1046-7'
def springer(pmid, url):
#     response = requests.get(url, headers=headers)
    text = requests.get(url, headers = headers,timeout=timeout).content.decode('utf-8')
    try:
        root = fromstring(text)
        # aff
        aff_index = 0
        for aff in root.cssselect('ul.author-affiliations li'):
            aff_index +=1
            for span in aff.findall('span'):
                if 'affiliation' in span.get('class'):
                    aff_name = span.text_content().strip()
                    cu.execute('insert into journal_aff (pmid, aff_index, aff_name) values(%s, %s, %s)', (pmid, aff_index, aff_name,))
                    con.commit()
        # get email list from the page footer     
        email_dict={}
        email_index = 0
        for author_li in root.cssselect('ul.authors li'):
            email_index += 1
            for a in author_li.findall('a'):
                if 'envelope' in a.get('class'):
#                     corresp = 1
                    email_e = a.get('title')
                    email_dict[str(email_index)] = email_e
        
        # get  author
#         print(email_dict)
        author_index=0
        for li in root.cssselect('ul.AuthorNames li'):
            if li.get('class') == 'AuthorNames_moreLink' or li.get('class') == 'AuthorNames_hideLink':
                continue
            corresp=0
            author=None
            email = None
            author_aff_name = None
            author_index +=1
            if li.cssselect('span.AuthorName') is not None:
#                 print(tostring(li))
                author = li.cssselect('span.AuthorName')[0].text.replace('&nbsp', ' ')
#                 print(author)
                if str(author_index) in email_dict:
                    email = email_dict[str(author_index)]
#                     print(email)
            li_a = li.find('a')
            if li_a is not None and 'mailto' in li_a.get('href'):
                corresp=1
                email = li_a.get('href').replace('mailto:', '')
            
            author_aff_list = li.cssselect('span.AuthorsName_affiliation span')
            for author_aff in author_aff_list:
                author_aff_name = author_aff.text_content()
                cu.execute('insert into journal_author values(%s, %s, %s, %s, %s, %s)',
                               (pmid, author_index, author, author_aff_name, email, corresp,))
                con.commit()
            
        # keywords
    #     keywords = root.cssselect('span.Keyword')
        for kw in root.cssselect('span.Keyword'):
            cu.execute('insert into journal_keyword values(%s, %s)', (pmid, kw.text_content().strip(),))
            con.commit()
    except Exception as e:
        with open('./errors/'+ str(pmid) + '.html', 'w') as f:
            f.write(text)
        cu.execute('insert into journal_error values(%s, %s)', (pmid, str(e),))
        con.commit()

    

In [9]:
# con = pymysql.connect(host='localhost', user='root', passwd='', db='xiaobaifinder', charset='utf8')
# cu = con.cursor()
# # cu_r = con.cursor()
# # cu_r.execute('select pmid,link from doi_link where link like "%wiley%" limit 100,10')
# # link_list = cu_r.fetchall()
# # for link in link_list:
# #     wiley(link[0], link[1])
# # print(link_list)
# # url = 'http://link.springer.com/article/10.1007%2Fs40261-015-0366-7'
# link = 'http://onlinelibrary.wiley.com/wol1/doi/10.1002/anie.201508505/abstract'
# wiley(1111, link)
# con.close()

In [10]:
def elsevier(pmid, url):
    response = requests.get(url,timeout=timeout)
#     server = response.headers['Server']
#     if 'sciencedirect' in  server:
    get_sciencedirect(pmid, response.content)
#     else:
#         cu.execute('insert into journal_error values(%s, %s)', (pmid, str(server)))
#         con.commit()


In [11]:
def get_sciencedirect(pmid, text):
    try:
        
        root = fromstring(text)
    #     ul = root.cssselect('ul.authorGroup')[0]
        # aff
        aff_dict = {}
        aff_index = 0
        for aff in root.cssselect('ul.affiliation li'):
            aff_index +=1
            aff_id = aff.get('id')
            aff_name = aff.find('span').text_content()
            aff_dict[aff_id] = str(aff_name)
            cu.execute('insert into journal_aff (pmid, aff_index, aff_name) values(%s, %s, %s)', (pmid, aff_index, aff_name,))
            con.commit()
        # author
        author_index = 0
        for author_li in root.cssselect('ul.authorGroup li'):
            a_list = author_li.findall('a')
            corresp=0
            author=None
            email = None
            author_aff_name = None
            author_index +=1
            author_aff_list = []
            for a in a_list:
                if 'authorName' in a.get('class'):
                    author = a.text_content()
                if a.get('title') and 'Corresponding author contact information' in a.get('title'):
                    corresp = 1
    #                 print(corresp)
                if a.get('class') and a.get('href') and 'auth_mail' in a.get('class'):
                    email = a.get('href').replace('mailto:', '')
    #                 print(email)
                if a.get('title') and 'Affiliation' in a.get('title') or len(aff_dict)==1:

                    if len(aff_dict) == 1:
    #                     author_aff_name=aff_dict[aff_id]
                        author_aff_list.append(aff_dict[aff_id])
                    else:
                        aff = a.get('href').replace('#', '')
    #                     author_aff_name = aff_dict[aff]
                        author_aff_list.append(aff_dict[aff])
            if author_aff_list:
                for author_aff_name in author_aff_list:
                    cu.execute('insert into journal_author values(%s, %s, %s, %s, %s, %s)',
                               (pmid, author_index, author, author_aff_name, email, corresp,))
                    con.commit()
            else:
                cu.execute('insert into journal_author values(%s, %s, %s, %s, %s, %s)',
                               (pmid, author_index, author, author_aff_name, email, corresp,))
                con.commit()

        #keywords
        keywords = root.cssselect('ul.keyword li')
        for kw in keywords:
            cu.execute('insert into journal_keyword values(%s, %s)', (pmid, kw.text_content().replace(';', '').strip(),))
            con.commit()
    except Exception as e:
        with open('./errors/'+ str(pmid) + '.html', 'w') as f:
            f.write(text)
        cu.execute('insert into journal_error values(%s, %s)', (pmid, str(e)))
        con.commit()
            

In [12]:
def get_website(url):
    netloc = urllib.parse.urlsplit(url).netloc
    out_split = netloc.split('.')
    if len(out_split)<3:
        out = web_site
    else:
        out = '.'.join(out_split[len(out_split)-2:])
    if out == '':
        out = netloc
    return out
def get_journal(pmid, link):
    try:
        if 'elsevier' in link or 'sciencedirect' in link:
            elsevier(pmid, link)
        elif 'springer' in link:
            springer(pmid, link)
        elif 'wiley' in link:
            wiley(pmid, link)
        elif 'acs.org' in link:
            acs(pmid, link)
        elif 'rsc.com' in link:
            rsc(pmid, link)
        elif 'tandfonline' in link:
            tandfonline(pmid, link)
        else:
            pass
    except:
        pass


In [13]:
con = pymysql.connect(host='localhost', user='root', passwd='', db='xiaobaifinder', charset='utf8')
cu = con.cursor()
# cu.execute('create table if not exists journal_keyword(pmid int, keywords varchar(200))')
# cu.execute('create table if not exists journal_aff(pmid int,aff_index int, aff_name Text)')
# cu.execute('create table if not exists journal_author(pmid int,author_index int,\
#             author varchar(200), author_aff Text, email varchar(200), corresp int)')
# cu.execute('create table if not exists journal_error(pmid int, errors Text)')
# con.commit()
cu_r = con.cursor()
cu = con.cursor()
cu_r.execute('select pmid,link from doi_link where pmid not in (select distinct pmid from journal_author)')
# link_list = cu_r.fetchall()


pool = multiprocessing.Pool(processes= 10) # 设置进程数
for ii in range(cu_r.rowcount):
    (pmid,link,) = cu_r.fetchone()
    pool.apply_async(get_journal, (int(pmid), link,))

pool.close()
pool.join()

cu.close()
con.close()

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/ipykernel/__main__.py:21: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/ipykernel/__main__.py:21: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/ipykernel/__main__.py:21: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/ipykernel/__main__.py:21: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
/Library/Frameworks/Python.framework/Ver

In [ ]:
def get_content(url):
    try:
        storage = io.BytesIO()
        c = pycurl.Curl()
        c.setopt(c.URL, url)
        c.setopt(pycurl.FOLLOWLOCATION, 1) 
        c.setopt(pycurl.WRITEFUNCTION, storage.write)
        # c.setopt(c.HTTPHEADER,["Accept: application/vnd.crossref.unixsd+xml"])
        c.perform()
        out = c.getinfo(pycurl.EFFECTIVE_URL)
#         cu_w.execute('insert into doi_link values(%s, %s, %s)', (pmid,doi, out))
#         con.commit()
        c.close()
        content = storage.getvalue().decode('utf-8')
#         print(out)
#         print(content)
        storage.close()
        return content,out
    except Exception as e:
        print(e)

In [ ]:
con = pymysql.connect(host='localhost', user='root', passwd='', db='xiaobaifinder', charset='utf8')
cu = con.cursor()
pmid ='123'
e='fdsd'
cu.execute('insert into journal_error values(%s, %s)', (pmid, str(e)))
con.commit()
con.commit()
cu.close()
con.close()